In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/MyDrive')


Mounted at /content/drive/


In [ ]:
cd CZ4034/

/content/drive/MyDrive/CZ4034


In [ ]:
import numpy as np
import regex as re
import nltk
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from time import time
import pandas as pd

In [ ]:
train_df = pd.read_csv('shared_data_train_subj.csv')
test_df = pd.read_csv('shared_data_test_subj.csv')
print(len(train_df[train_df.label == 0]))
print(len(train_df[train_df.label == 1]))
print(len(test_df[test_df.label == 0]))
print(len(test_df[test_df.label == 1]))

390
390
110
110


In [ ]:
import re

def alphanum_cleaning(x):
    x = str(x)
    pattern = '[^a-zA-Z\ ]'
    x = re.sub(pattern,'',x)
    x = x.lower()
    x = x.strip()
    return x

train_df['text'] = train_df['text'].apply(alphanum_cleaning)
test_df['text'] = test_df['text'].apply(alphanum_cleaning)

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from sklearn.model_selection import train_test_split, KFold

col_selected = 'text'



x_train, y_train = train_df[col_selected], train_df['label']
x_test, y_test = test_df[col_selected], test_df['label']


# embedding
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
 
def get_sentence_vec(datasets):

    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(datasets)]
    model = Doc2Vec(vector_size=300, dm=1, window=4, min_count=1, epochs=50)
    model.build_vocab(documents)
    model.train(documents,total_examples=model.corpus_count,epochs=model.epochs)

    model.save('doc2vec_subjectivity_model.pkl') 
    # get sent vector
    documents_vecs = np.concatenate([np.array(model.docvecs[sen.tags[0]].reshape(1, 300)) for sen in documents])
    return documents_vecs

split_text = lambda x: str(x).split()
x_train_words = x_train.apply(split_text)

# train the model using x_train
documents_vec=get_sentence_vec(x_train_words)
doc2vec_model=Doc2Vec.load('doc2vec_subjectivity_model.pkl')



In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
nfolds=5

Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.05, 0.1, 0.15, 0.20, 0.25]
degrees = [0, 1, 2, 3, 4, 5, 6]
kernels = ['rbf', 'linear', 'poly']
param_grid = {'C': Cs, 'gamma' : gammas, 'degree' : degrees, 'kernel' : kernels}

grid_search = GridSearchCV(svm.SVC(), param_grid, cv = nfolds)
grid_search.fit(documents_vec, y_train)
grid_search.best_params_


{'C': 1, 'degree': 2, 'gamma': 0.2, 'kernel': 'poly'}

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
SVM = svm.SVC(**grid_search.best_params_).fit(documents_vec, y_train)

start = time()
test_vector = list()
x_test_words = x_test.apply(split_text)
for sent in x_test_words:
  test_vector.append(doc2vec_model.infer_vector(sent))
result = SVM.predict(test_vector)
end = time()
confuse = metrics.confusion_matrix(y_test, result)


F1_score = metrics.f1_score(y_test, result)
average_precision = metrics.average_precision_score(y_test, result)
precision_score= metrics.precision_score(y_test, result)
recall_score = metrics.recall_score(y_test, result)

print(f'=========={col_selected}==========')
print(f'accuracy: {accuracy_score(y_test, result)}')
print('F1 score: {0:0.3f}'.format(F1_score))
print('Precision score: {0:0.3f}'.format(precision_score))
print('Recall score: {0:0.3f}'.format(recall_score))
print('Average precision-recall score: {0:0.3f}'.format(
      average_precision))
print(f"Runtime of the program is {end - start}, number of record classified is {len(test_vector)} \naverage number of record per second is {len(test_vector)/(end-start)}")



==========text==========
accuracy: 0.6090909090909091
F1 score: 0.636
Precision score: 0.595
Recall score: 0.682
Average precision-recall score: 0.565
Runtime of the program is 1.664261817932129, number of record classified is 220 
average number of record per second is 132.19073923918617


In [ ]:
from xgboost import XGBClassifier

# A parameter grid for XGBoost
# can add more range
params = {
    'eta': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.5, 0.7, 1.0],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'min_child_weight': [1, 5, 10],
}

xgb = XGBClassifier()

folds = 5

grid = GridSearchCV(estimator=xgb, param_grid=params, scoring='accuracy', cv=folds, verbose=3)
grid.fit(documents_vec, y_train)

Fitting 3 folds for each of 243 candidates, totalling 729 fits
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.5 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.5, score=0.608, total=   0.6s
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV]  colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.5, score=0.642, total=   0.5s
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.5 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.0s remaining:    0.0s


[CV]  colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.5, score=0.608, total=   0.5s
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.7 
[CV]  colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.7, score=0.662, total=   0.5s
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.7 
[CV]  colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.7, score=0.581, total=   0.5s
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.7 
[CV]  colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.7, score=0.608, total=   0.5s
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=1.0 
[CV]  colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=1.0, score=0.615, total=   0.6s
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=1.0 
[CV]  cols

[Parallel(n_jobs=1)]: Done 729 out of 729 | elapsed: 10.1min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={'colsample_bytree': [0.5, 0.7, 1.0],
                         'eta': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7],
                         'min_child_weigh

In [ ]:
xgb = XGBClassifier(**grid.best_params_)
xgb.fit(documents_vec, y_train)


start = time()
test_vector = list()
x_test_words = x_test.apply(split_text)
for sent in x_test_words:
  test_vector.append(doc2vec_model.infer_vector(sent))
result = xgb.predict(test_vector)
end = time()

confuse = metrics.confusion_matrix(y_test, result)

F1_score = metrics.f1_score(y_test, result)
average_precision = metrics.average_precision_score(y_test, result)
precision_score= metrics.precision_score(y_test, result)
recall_score = metrics.recall_score(y_test, result)

print(f'=========={col_selected}==========')
print(f'accuracy: {accuracy_score(y_test, result)}')
print('F1 score: {0:0.3f}'.format(F1_score))
print('Precision score: {0:0.3f}'.format(precision_score))
print('Recall score: {0:0.3f}'.format(recall_score))
print('Average precision-recall score: {0:0.3f}'.format(
      average_precision))
print(f"Runtime of the program is {end - start}, number of record classified is {len(test_vector)} \naverage number of record per second is {len(test_vector)/(end-start)}")

==========text==========
accuracy: 0.6181818181818182
F1 score: 0.641
Precision score: 0.605
Recall score: 0.682
Average precision-recall score: 0.571
Runtime of the program is 1.7171554565429688, number of record classified is 220 
average number of record per second is 128.118860270759
